In [0]:
MERGE INTO IDENTIFIER(:catalog || '.' || :schema || '.classified') AS target
USING (
  SELECT 
    e.path,
    e.key_information,
    e.preamble,
    array_agg(vs.key_information) AS vector_search_results,
    AI_QUERY(
        "databricks-claude-sonnet-4-5",
        CONCAT(
          'You are a contract document classification and extraction agent. 
          
          Given a master agreement document (with its preamble and key information) and a set of related documents (vector search results), classify each related document as one of: Amendment, RateSheet, OtherRelatedFile, or Unclassified. 
          
          Use the content of the preamble and key information from the master agreement, and the full text and key information from each related document for classification. 
          
          For amendments, determine their sequence (e.g., Amendment 1, Amendment 2) using the date found inside the document text (not file name or metadata) to establish chronological order. Highlight the latest amendment for each master agreement.
          
          Return ONLY valid JSON with this structure: {"MasterAgreements":[{"FileName":"...","RelatedDocuments":{"Amendments":[{"FileName":"...","Date":"...","Sequence":"..."}],"LatestAmendment":{"FileName":"...","Date":"...","Sequence":"..."},"RateSheets":["..."],"OtherRelatedFiles":["..."]}}],"UnclassifiedFiles":[{"FileName":"...","Title":"..."}]} 
          
          Use internal document content for classification, not file names. 
          Ignore duplicates. 
          Flag uncertain documents as UnclassifiedFiles with the full file name from the key information',
          'Master Agreement Path: ', e.path, '\n',
          'Master Key Information: ', e.key_information, '\n',
          'Master Preamble: ', e.preamble, '\n',
          'Related Documents: ', array_join(array_agg(vs.key_information), '\n---\n'), '\n',
          'Vendor Files:', array_join(flatten(collect_list(vs.other_vendor_files)), ','), '\n',
          'Vendors', array_join(array_agg(vs.vendor_name), ', '), '\n'
          ),
      responseFormat => 'STRUCT<classification:STRUCT<MasterAgreements:ARRAY<STRUCT<FileName:STRING, RelatedDocuments:STRUCT<Amendments:ARRAY<STRUCT<FileName:STRING, Date:STRING, Sequence:STRING>>, LatestAmendment:STRUCT<FileName:STRING, Date:STRING, Sequence:STRING>, RateSheets:ARRAY<STRING>, OtherRelatedFiles:ARRAY<STRING>>>>, UnclassifiedFiles:ARRAY<STRUCT<FileName:STRING, Title:STRING>>>>'
    ) AS classification
  FROM shm.contracts.extracted e
  LEFT JOIN LATERAL (
    SELECT * FROM vector_search(
      index => :catalog || '.' || :schema || '.index',
      query_text => e.key_information,
      query_type => 'HYBRID',
      num_results => 20
    )
  ) vs ON TRUE
  GROUP BY e.path, e.key_information, e.preamble
) AS source
ON target.path = source.path
WHEN NOT MATCHED THEN
  INSERT (
    path,
    key_information,
    preamble,
    vector_search_results,
    classification
  )
  VALUES (
    source.path,
    source.key_information,
    source.preamble,
    source.vector_search_results,
    source.classification
  )